# 🤖 Modeling Template
Notebook ini digunakan untuk membangun dan mengevaluasi model Machine Learning/Data Mining.

## Langkah yang disarankan:
- Pisahkan data latih dan data uji
- Latih model (misalnya: Decision Tree, Random Forest, SVM, dll.)
- Evaluasi menggunakan metrik: accuracy, precision, recall, F1-score, ROC, confusion matrix

In [1]:
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
import matplotlib.pyplot as plt
import networkx as nx
from itertools import combinations
from collections import Counter
import seaborn as sns
import ast
import numpy as np


In [ ]:
# Menjalankan algoritma Apriori pada data transaksi yang sudah di-one-hot encode
# min_support=0.002 berarti hanya itemset yang muncul di minimal 0.2% dari total transaksi yang akan diambil
# use_colnames=True agar hasilnya menampilkan nama item (produk) asli, bukan index kolom
frequent_itemsets = apriori(df_encoded, min_support=0.002, use_colnames=True)

# Menampilkan itemset-itemset yang memenuhi syarat minimum support
print("Frequent Itemsets:")
print(frequent_itemsets)

In [ ]:
# Menghasilkan aturan asosiasi dari frequent itemsets
# Menggunakan metrik 'confidence' dan ambang batas minimum confidence sebesar 0.05 (5%)
rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.05)

# Menambahkan kolom baru yang menunjukkan panjang (jumlah item) dari antecedents dan consequents
# Gunanya untuk filter atau analisis lebih lanjut, misalnya mencari aturan 1 → 1
rules['antecedent_len'] = rules['antecedents'].apply(lambda x: len(x))
rules['consequent_len'] = rules['consequents'].apply(lambda x: len(x))

# Menampilkan 10 aturan asosiasi pertama dengan kolom-kolom penting
print("\n🔎 Semua Aturan (tanpa filter 1→1):")
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(10))

# Menampilkan jumlah total aturan yang dihasilkan
print(f"Jumlah total aturan: {len(rules)}")


🔎 Semua Aturan (tanpa filter 1→1):
       antecedents   consequents   support  confidence      lift
0  (Air Freshener)  (Toothpaste)  0.002363    0.065157  0.918370
1          (Apple)  (Toothpaste)  0.002368    0.066241  0.933649
2      (BBQ Sauce)  (Toothpaste)  0.002231    0.061893  0.872361
3     (Baby Wipes)  (Toothpaste)  0.002396    0.066195  0.932996
4         (Banana)  (Toothpaste)  0.002313    0.063853  0.899980
5    (Bath Towels)  (Toothpaste)  0.002276    0.063002  0.887986
6           (Beef)  (Toothpaste)  0.002305    0.063778  0.898927
7          (Bread)  (Toothpaste)  0.002272    0.063538  0.895548
8          (Broom)  (Toothpaste)  0.002290    0.064008  0.902164
9         (Butter)  (Toothpaste)  0.002241    0.062528  0.881308
Jumlah total aturan: 80


In [ ]:
# Filter aturan asosiasi yang hanya terdiri dari 1 item di antecedent dan 1 item di consequent
# Tujuannya untuk fokus pada aturan sederhana dan mudah diinterpretasikan
rules_1to1 = rules[
    (rules['antecedent_len'] == 1) &  # Hanya yang antecedent-nya 1 item
    (rules['consequent_len'] == 1)   # Dan consequent-nya juga 1 item
]

# Menampilkan aturan-aturan hasil filter 1→1
print("\n📌 Aturan 1 item → 1 item:")
print(rules_1to1[['antecedents', 'consequents', 'support', 'confidence', 'lift']])

# Menampilkan jumlah total aturan 1→1 yang ditemukan
print(f"Jumlah aturan 1→1 ditemukan: {len(rules_1to1)}")



📌 Aturan 1 item → 1 item:
         antecedents   consequents   support  confidence      lift
0    (Air Freshener)  (Toothpaste)  0.002363    0.065157  0.918370
1            (Apple)  (Toothpaste)  0.002368    0.066241  0.933649
2        (BBQ Sauce)  (Toothpaste)  0.002231    0.061893  0.872361
3       (Baby Wipes)  (Toothpaste)  0.002396    0.066195  0.932996
4           (Banana)  (Toothpaste)  0.002313    0.063853  0.899980
..               ...           ...       ...         ...       ...
75            (Tuna)  (Toothpaste)  0.002303    0.064061  0.902919
76  (Vacuum Cleaner)  (Toothpaste)  0.002227    0.061908  0.872564
77         (Vinegar)  (Toothpaste)  0.002253    0.062903  0.886596
78           (Water)  (Toothpaste)  0.002232    0.062582  0.882075
79          (Yogurt)  (Toothpaste)  0.002323    0.064251  0.905596

[80 rows x 5 columns]
Jumlah aturan 1→1 ditemukan: 80


In [ ]:
# Filter aturan asosiasi 1 → 1 (satu produk ke satu produk)
rules_1to1 = rules[
    (rules['antecedent_len'] == 1) &  # Antecedent hanya 1 item
    (rules['consequent_len'] == 1)   # Consequent juga hanya 1 item
]

# Hitung TCR (Total Contribution Ratio)
# TCR di sini adalah total nilai confidence dari semua aturan yang berasal dari produk tertentu sebagai antecedent
# Langkah pertama: ambil nama produk dari set antecedent
tcr = rules_1to1.groupby(rules_1to1['antecedents'].apply(lambda x: list(x)[0]))['confidence'].sum().reset_index()

# Ganti nama kolom agar lebih deskriptif
tcr.columns = ['product', 'TCR']

# Urutkan dari produk dengan TCR tertinggi
tcr = tcr.sort_values(by='TCR', ascending=False)

# Tampilkan hasil TCR
print("📊 Total Contribution Ratio (TCR):")
print(tcr)


📊 Total Contribution Ratio (TCR):
          product       TCR
71       Tomatoes  0.066959
46         Onions  0.066363
14         Cheese  0.066324
20      Deodorant  0.066294
1           Apple  0.066241
..            ...       ...
55   Power Strips  0.061570
64           Soda  0.061530
15        Chicken  0.060993
11        Carrots  0.060893
60  Shaving Cream  0.060405

[80 rows x 2 columns]


In [ ]:
# Filter aturan 1→1 dengan kriteria kuat:
# Confidence harus lebih dari 0.05 (5%) dan lift lebih dari 0.8
# Artinya kita hanya ambil aturan yang cukup kuat dan memiliki asosiasi positif
arc = rules_1to1[
    (rules_1to1['confidence'] > 0.05) & 
    (rules_1to1['lift'] > 0.8)
]

# Ambil kolom penting saja untuk visualisasi atau analisis jaringan
arc_chart = arc[['antecedents', 'consequents', 'confidence', 'lift']].copy()

# Karena antecedents dan consequents masih dalam bentuk set, kita ambil elemen pertama (hanya satu karena aturan 1→1)
arc_chart['antecedents'] = arc_chart['antecedents'].apply(lambda x: list(x)[0])
arc_chart['consequents'] = arc_chart['consequents'].apply(lambda x: list(x)[0])

# Tampilkan hasil tabel adjacency yang siap digunakan untuk visualisasi grafik jaringan (network chart)
print("📐 Adjacency Relation Chart (ARC):")
print(arc_chart)


📐 Adjacency Relation Chart (ARC):
       antecedents consequents  confidence      lift
0    Air Freshener  Toothpaste    0.065157  0.918370
1            Apple  Toothpaste    0.066241  0.933649
2        BBQ Sauce  Toothpaste    0.061893  0.872361
3       Baby Wipes  Toothpaste    0.066195  0.932996
4           Banana  Toothpaste    0.063853  0.899980
..             ...         ...         ...       ...
75            Tuna  Toothpaste    0.064061  0.902919
76  Vacuum Cleaner  Toothpaste    0.061908  0.872564
77         Vinegar  Toothpaste    0.062903  0.886596
78           Water  Toothpaste    0.062582  0.882075
79          Yogurt  Toothpaste    0.064251  0.905596

[80 rows x 4 columns]


🧠 Tahap Modeling – Langkah-Langkah
1. Mencari Frequent Itemsets dengan Apriori
- Menggunakan algoritma Apriori dari mlxtend untuk menemukan kombinasi produk yang sering dibeli bersamaan.
- Ditetapkan nilai min_support = 0.002, artinya hanya itemset yang muncul di minimal 0.2% transaksi yang akan dipertimbangkan.

2. Membangun Aturan Asosiasi (Association Rules)
- Aturan dibentuk dari itemset menggunakan metrik confidence sebagai dasar, dengan min_threshold = 0.05 (5%).
- Output berupa kombinasi antecedent → consequent dengan nilai support, confidence, dan lift.

3. Menyaring Aturan 1 → 1
- Diambil hanya aturan yang memiliki 1 produk di antecedent dan 1 produk di consequent, agar lebih mudah diinterpretasi dan lebih relevan untuk rekomendasi sederhana.

4.  Menghitung Total Contribution Ratio (TCR)
- TCR dihitung dengan menjumlahkan seluruh nilai confidence dari aturan yang memiliki produk tertentu sebagai antecedent.
- Produk dengan TCR tinggi dianggap memiliki kontribusi besar dalam memicu pembelian produk lain.

5. Membentuk Adjacency Relation Chart (ARC)
- Disaring aturan 1→1 yang memiliki confidence > 0.05 dan lift > 0.8, menandakan hubungan yang kuat dan bermakna.
- Format aturan diubah menjadi tabel adjacency (arah → antar produk) yang cocok digunakan untuk visualisasi graf jaringan.